# 1. Cargue sus datos

Por favor cargue los datos en la siguiente sección

## 1.1 Objetivos

- Objetivo general
- Objetivos especificos

## 1.2 Hipótesis

# 1.3. ¿Qué pregunta(s) quiere responder a partir de sus datos?

El proyecto de este modulo es muy amplio, debido a las diferentes arquitecturas de bases de datos presentadas y a sus deseos.

- Sqlite

- Postgresql

- Cassandra

- MongoDB

- Dask

- Spark

# Sqlite
---

- Cargar datos de distinta naturaleza (4 o 5 tablas), con interrelaciones
- Distintos tipos de consulta



### Sencillos
- Cargar datos
- Análisis de datos
- Consultas básicas
- Consultas avanzadas

### Medios
- Cargar datos en sqlite y enviarlos a mongodb o a PySpark a un RDD
- Realizar al menos unas 3 o 4 consultas distintas utilizando `common table expressions (CTEs) with o subqueries`
- Crear indices en distintas tablas explicando porque son necesarios

### Avanzados
- Enviar datos a Apache Kafka y serializarlos a sqlite utilizando Python
- Cargar datos en sqlite y hacer machine learning en Dask o PySpark utilizando archivos parquet o csv

# Postgresql (postgis análisis geoespacial y qgis)
---

- Cargar datos de distinta naturaleza
- Distintios tipos de consulta
- Clara definición de datos
- Resolver la hipótesis que se esta planteando

## Ejemplos


### Sencillos
- Cargar datos
- Análisis de datos
- Consultas básicas
- Consultas avanzadas

### Medios
- Cargar datos en postgresql y enviarlos a mongodb o a PySpark a un RDD
- Cargar datos en formato JSONB en postgresql
- Crear indices en distintas tablas explicando porque son necesarios
- Hacer consultas en formato JSONB en postgresql
- Realizar al menos unas 3 o 4 consultas distintas utilizando `common table expressions (CTEs)` o subqueries

### Avanzados
- Crear un ambiente principal-secundario en postgresql
- Enviar datos a Kafka y serializarlos a postgresql utilizando Python
- Cargar datos en Postgresql y hacer machine learning en Dask o PySpark

# MongoDB
---

- Realizar distintos tipos de agregación
- Realizar distintos tipos de agrupamiento
- Plantear el modelo `query first design`
-


## Ejemplos

### Sencillos
- Cargar datos
- Análisis de datos
- Consultas básicas
- Consultas avanzadas

### Medios
- Cargar datos en postgresql y enviarlos a mongodb
- Cargar datos en MongoDB y procesarlos usando Dask o usando PySpark
- Realizar al menos unas 3 o 4 consultas distintas utilizando map-reduce

### Avanzados
- Docker para hacer un ambiente distribuido
- Con RaspberryPI crear un cluster de datos
- Cargar datos en MongoDB y hacer machine learning en Dask o PySpark

# Dask
---


## Ejemplos

### Sencillos
- Cargar datos de csv, json, xml, etc.
- Realizar un procesamiento distribuido utilizan n-jobs
- Crear un par de consultas que involucren particiones de datos con un grafo de al menos dos o tres niveles

### Medios
- Cargar datos en postgresql y procesarlos usando dask.
- Cargar datos en MongoDB y procesarlos usando Dask
- Comparar los resultados de hacer machine learning con SkLearn vs hacerlo con Dask-ML

### Avanzados
- Docker para hacer un ambiente distribuido
- Con RaspberryPI crear un cluster de datos y analizar los datos con Dask
- Cargar datos en MongoDB y hacer machine learning en Dask

# PySpark
---


## Ejemplos

### Sencillos
- Cargar datos de csv, json, xml, etc.
- Realizar un procesamiento distribuido utilizan n-jobs
- Crear un par de consultas que involucren particiones de datos

### Medios
- Cargar datos en postgresql y procesarlos usando Spark.
- Cargar datos en MongoDB y procesarlos usando Spark.
- Comparar los resultados de hacer machine learning con SkLearn vs hacerlo con PySpark (entender las limitaciones y usos de aprendizaje distribuido)

### Avanzados
- Docker para hacer un ambiente distribuido (super avanzado)
- Con RaspberryPI crear un cluster de datos y analizar los datos con Dask
- Cargar datos en MongoDB y hacer machine learning en PySpark